# Rekurrente neurale netværk

I det forrige modul har vi brugt rige semantiske repræsentationer af tekst og en simpel lineær klassifikator oven på embeddings. Denne arkitektur fanger den samlede betydning af ordene i en sætning, men den tager ikke højde for **rækkefølgen** af ordene, fordi aggregeringsoperationen oven på embeddings fjerner denne information fra den oprindelige tekst. Da disse modeller ikke kan modellere ords rækkefølge, kan de ikke løse mere komplekse eller tvetydige opgaver som tekstgenerering eller besvarelse af spørgsmål.

For at fange betydningen af tekstsekvenser skal vi bruge en anden neural netværksarkitektur, som kaldes et **rekurrent neuralt netværk**, eller RNN. I RNN sender vi vores sætning gennem netværket én symbol ad gangen, og netværket producerer en **tilstand**, som vi derefter sender til netværket igen sammen med det næste symbol.

Givet inputsekvensen af tokens $X_0,\dots,X_n$, skaber RNN en sekvens af neurale netværksblokke og træner denne sekvens end-to-end ved hjælp af backpropagation. Hver netværksblok tager et par $(X_i,S_i)$ som input og producerer $S_{i+1}$ som resultat. Den endelige tilstand $S_n$ eller output $X_n$ sendes til en lineær klassifikator for at producere resultatet. Alle netværksblokke deler de samme vægte og trænes end-to-end ved hjælp af én backpropagation-pass.

Fordi tilstandsvektorerne $S_0,\dots,S_n$ sendes gennem netværket, er det i stand til at lære de sekventielle afhængigheder mellem ord. For eksempel, når ordet *ikke* optræder et sted i sekvensen, kan det lære at negere visse elementer inden for tilstandsvektoren, hvilket resulterer i negation.

> Da vægtene for alle RNN-blokke på billedet er delt, kan det samme billede repræsenteres som én blok (til højre) med en rekurrent feedback-loop, som sender netværkets outputtilstand tilbage til input.

Lad os se, hvordan rekurrente neurale netværk kan hjælpe os med at klassificere vores nyheds-datasæt.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Enkel RNN-klassifikator

I tilfælde af en simpel RNN er hver rekurrent enhed et simpelt lineært netværk, som tager en sammenkædet inputvektor og tilstandsvektor og producerer en ny tilstandsvektor. PyTorch repræsenterer denne enhed med klassen `RNNCell`, og et netværk af sådanne celler - som laget `RNN`.

For at definere en RNN-klassifikator vil vi først anvende et indlejringslag for at reducere dimensionaliteten af inputvokabularet, og derefter placere et RNN-lag ovenpå:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Bemærk:** Vi bruger her et utrænet embedding-lag for enkelhedens skyld, men for endnu bedre resultater kan vi bruge et forudtrænet embedding-lag med Word2Vec- eller GloVe-embeddings, som beskrevet i den foregående enhed. For bedre forståelse kan du overveje at tilpasse denne kode til at arbejde med forudtrænede embeddings.

I vores tilfælde vil vi bruge en polstret data-loader, så hver batch vil indeholde et antal polstrede sekvenser af samme længde. RNN-laget vil tage sekvensen af embedding-tensorer og producere to outputs:
* $x$ er en sekvens af RNN-cellens outputs ved hvert trin
* $h$ er den endelige skjulte tilstand for det sidste element i sekvensen

Vi anvender derefter en fuldt forbundet lineær klassifikator for at få antallet af klasser.

> **Bemærk:** RNN'er er ret svære at træne, fordi når RNN-cellerne udfoldes langs sekvensens længde, bliver antallet af lag, der er involveret i backpropagation, ret stort. Derfor skal vi vælge en lille læringsrate og træne netværket på et større datasæt for at opnå gode resultater. Det kan tage ret lang tid, så det anbefales at bruge GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Long Short Term Memory (LSTM)

Et af de største problemer med klassiske RNN'er er det såkaldte **vanishing gradients**-problem. Fordi RNN'er trænes end-to-end i én backpropagation-pass, har de svært ved at propagere fejl til de første lag i netværket, og derfor kan netværket ikke lære relationer mellem fjerne tokens. En af måderne at undgå dette problem på er at introducere **eksplicit tilstandsadministration** ved at bruge såkaldte **gates**. Der er to mest kendte arkitekturer af denne type: **Long Short Term Memory** (LSTM) og **Gated Relay Unit** (GRU).

![Billede, der viser et eksempel på en long short term memory-celle](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM-netværket er organiseret på en måde, der ligner RNN, men der er to tilstande, der bliver sendt fra lag til lag: den faktiske tilstand $c$ og den skjulte vektor $h$. Ved hver enhed bliver den skjulte vektor $h_i$ sammenkædet med input $x_i$, og de styrer, hvad der sker med tilstanden $c$ via **gates**. Hver gate er et neuralt netværk med sigmoid-aktivering (output i intervallet $[0,1]$), som kan betragtes som en bitmaske, når den multipliceres med tilstandsvektoren. Der er følgende gates (fra venstre til højre på billedet ovenfor):
* **forget gate** tager den skjulte vektor og bestemmer, hvilke komponenter af vektoren $c$ vi skal glemme, og hvilke vi skal lade passere.
* **input gate** tager noget information fra input og den skjulte vektor og indsætter det i tilstanden.
* **output gate** transformerer tilstanden via et lineært lag med $\tanh$-aktivering og vælger derefter nogle af dens komponenter ved hjælp af den skjulte vektor $h_i$ for at producere den nye tilstand $c_{i+1}$.

Komponenterne i tilstanden $c$ kan betragtes som nogle flag, der kan tændes og slukkes. For eksempel, når vi støder på et navn som *Alice* i sekvensen, vil vi måske antage, at det refererer til en kvindelig karakter og hæve flaget i tilstanden, der indikerer, at vi har et kvindeligt substantiv i sætningen. Når vi senere støder på frasen *and Tom*, vil vi hæve flaget, der indikerer, at vi har et flertal substantiv. Ved at manipulere tilstanden kan vi således formodentlig holde styr på grammatiske egenskaber ved sætningsdele.

> **Note**: En fremragende ressource til at forstå LSTM's interne funktioner er denne fantastiske artikel [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) af Christopher Olah.

Selvom den interne struktur af en LSTM-celle kan virke kompleks, skjuler PyTorch denne implementering inde i `LSTMCell`-klassen og tilbyder `LSTM`-objektet til at repræsentere hele LSTM-laget. Implementeringen af en LSTM-klassifikator vil derfor være ret lig den simple RNN, som vi har set ovenfor:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Pakkede sekvenser

I vores eksempel var vi nødt til at udfylde alle sekvenser i minibatchen med nulvektorer. Selvom det medfører noget spild af hukommelse, er det mere kritisk med RNN'er, at der oprettes ekstra RNN-celler for de udfyldte inputelementer, som deltager i træningen, men ikke indeholder nogen vigtig inputinformation. Det ville være meget bedre kun at træne RNN til den faktiske sekvensstørrelse.

For at gøre dette introduceres et specielt format til opbevaring af udfyldte sekvenser i PyTorch. Antag, at vi har en udfyldt minibatch, der ser sådan ud:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Her repræsenterer 0 udfyldte værdier, og den faktiske længdevektor for inputsekvenserne er `[5,3,1]`.

For effektivt at træne RNN med udfyldte sekvenser ønsker vi at starte træningen af den første gruppe af RNN-celler med en stor minibatch (`[1,6,9]`), men derefter afslutte behandlingen af den tredje sekvens og fortsætte træningen med kortere minibatches (`[2,7]`, `[3,8]`) osv. Således repræsenteres den pakkede sekvens som én vektor - i vores tilfælde `[1,6,9,2,7,3,8,4,5]`, og længdevektoren (`[5,3,1]`), hvorfra vi nemt kan rekonstruere den oprindelige udfyldte minibatch.

For at producere en pakket sekvens kan vi bruge funktionen `torch.nn.utils.rnn.pack_padded_sequence`. Alle rekurrente lag, inklusive RNN, LSTM og GRU, understøtter pakkede sekvenser som input og producerer pakket output, som kan dekodes ved hjælp af `torch.nn.utils.rnn.pad_packed_sequence`.

For at kunne producere en pakket sekvens skal vi give længdevektoren til netværket, og derfor har vi brug for en anden funktion til at forberede minibatches:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Den faktiske netværk vil være meget lig `LSTMClassifier` ovenfor, men `forward`-passet vil modtage både den polstrede minibatch og vektoren af sekvenslængder. Efter at have beregnet embedding, beregner vi den pakkede sekvens, sender den til LSTM-laget og pakker derefter resultatet ud igen.

> **Note**: Vi bruger faktisk ikke det udpakkede resultat `x`, fordi vi bruger output fra de skjulte lag i de følgende beregninger. Derfor kan vi fjerne udpakkingen helt fra denne kode. Grunden til, at vi placerer det her, er for at gøre det nemt for dig at ændre denne kode, hvis du skulle få brug for at bruge netværkets output i yderligere beregninger.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Bemærk:** Du har måske bemærket parameteren `use_pack_sequence`, som vi sender til træningsfunktionen. I øjeblikket kræver funktionen `pack_padded_sequence`, at længdesekvenstensoren er på CPU-enheden, og derfor skal træningsfunktionen undgå at flytte længdesekvensdataene til GPU under træning. Du kan se implementeringen af funktionen `train_emb` i filen [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## Bidirektionelle og flerlags RNN'er

I vores eksempler har alle rekurrente netværk opereret i én retning, fra starten af en sekvens til slutningen. Det virker naturligt, fordi det minder om den måde, vi læser og lytter til tale. Men i mange praktiske tilfælde har vi tilfældig adgang til inputsekvensen, og det kan derfor give mening at udføre rekurrent beregning i begge retninger. Sådanne netværk kaldes **bidirektionelle** RNN'er, og de kan oprettes ved at angive parameteren `bidirectional=True` til RNN/LSTM/GRU-konstruktøren.

Når vi arbejder med et bidirektionelt netværk, har vi brug for to skjulte tilstandsvektorer, én for hver retning. PyTorch koder disse vektorer som én vektor med dobbelt så stor størrelse, hvilket er ret praktisk, da man normalt vil sende den resulterende skjulte tilstand til et fuldt forbundet lineært lag. Man skal blot tage denne forøgelse i størrelse i betragtning, når man opretter laget.

Et rekurrent netværk, enten én-retnings eller bidirektionelt, fanger visse mønstre inden for en sekvens og kan gemme dem i en tilstandsvektor eller sende dem videre til output. Ligesom med konvolutionelle netværk kan vi bygge et andet rekurrent lag oven på det første for at fange mønstre på et højere niveau, bygget fra lav-niveau mønstre, som det første lag har udtrukket. Dette fører os til begrebet **flerlags RNN**, som består af to eller flere rekurrente netværk, hvor output fra det foregående lag sendes til det næste lag som input.

![Billede der viser en flerlags lang-kort-tids-hukommelses-RNN](../../../../../translated_images/multi-layer-lstm.dd975e29bb2a59fe58b429db833932d734c81f211cad2783797a9608984acb8c.da.jpg)

*Billede fra [denne fantastiske artikel](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) af Fernando López*

PyTorch gør det nemt at konstruere sådanne netværk, da man blot skal angive parameteren `num_layers` til RNN/LSTM/GRU-konstruktøren for automatisk at bygge flere lag af rekurrence. Dette betyder også, at størrelsen på den skjulte/tilstandsvektor vil stige proportionalt, og man skal tage dette i betragtning, når man håndterer output fra de rekurrente lag.


## RNN'er til andre opgaver

I denne enhed har vi set, at RNN'er kan bruges til sekvensklassifikation, men faktisk kan de håndtere mange flere opgaver, såsom tekstgenerering, maskinoversættelse og mere. Vi vil se nærmere på disse opgaver i den næste enhed.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
